## OBJECTIVES

- Predict CVD and cancer caused mortality rates in England using air quality data available from Copernicus Atmosphere Monitoring Service
- The goal is to predict mortality rates (number of deaths per 100,000 people) for each English region using daily average measurements of ozone (O3), nitrogen dioxide (NO2), PM10 (particulate matter with diameter less than or equal to 10 micrometers), PM2.5 (diameter 2.5 micrometers or less) and temperature, data freely available from Copernicus Atmosphere Monitoring Service.

Poor air quality is a significant public health issue. The burden of particulate air pollution in the UK in 2008 was estimated to be equivalent to nearly 29,000 deaths at typical ages and an associated loss of population life of 340,000 life years lost.

In [1]:
import pandas as pd

#loading data

train  = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
test

,Id,region,date,O3,PM10,PM25,NO2,T2M
0,18404,E12000006,2012-05-28,75.980,20.876,19.123,9.713,290.787
1,18405,E12000006,2012-05-29,73.084,21.660,17.794,8.417,288.474
2,18406,E12000006,2012-05-30,59.350,21.925,17.699,10.878,289.889
3,18407,E12000006,2012-05-31,45.991,14.549,11.386,10.302,287.815
4,18408,E12000006,2012-06-01,52.210,11.208,9.545,8.598,287.627
...,...,...,...,...,...,...,...,...
7881,26285,E12000009,2014-12-27,57.951,6.422,3.683,6.082,277.350
7882,26286,E12000009,2014-12-28,41.479,14.802,10.989,14.361,275.209
7883,26287,E12000009,2014-12-29,32.742,17.722,12.177,17.386,275.772
7884,26288,E12000009,2014-12-30,39.006,16.997,10.754,15.317,276.465


In [19]:
#let's review our main dataset
train.info()

# RESULT: the train data has 18403 rows and 9 columns. 
# Out of these 9 columns, 6 have float64 class, 1 data object, 1 integer and 1 object
# *we can do a similar check for test data. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18403 entries, 0 to 18402
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Id              18403 non-null  int64         
 1   region          18403 non-null  object        
 2   date            18403 non-null  datetime64[ns]
 3   mortality_rate  18403 non-null  float64       
 4   O3              18403 non-null  float64       
 5   PM10            18403 non-null  float64       
 6   PM25            18403 non-null  float64       
 7   NO2             18403 non-null  float64       
 8   T2M             18403 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 1.3+ MB


In [20]:
train.describe()

,Id,date,mortality_rate,O3,PM10,PM25,NO2,T2M
count,18403.000000,18403,18403.000000,18403.000000,18403.000000,18403.000000,18403.000000,18403.000000
mean,9202.000000,2009-10-26 20:53:27.390099200,1.301737,45.325857,13.712272,7.495804,12.038705,283.002235
min,1.000000,2007-01-02 00:00:00,0.439000,0.988000,2.020000,0.904000,1.104000,265.562000
25%,4601.500000,2008-05-27 00:00:00,1.102000,35.078500,8.658500,4.028000,7.337000,279.321500
50%,9202.000000,2009-10-20 00:00:00,1.281000,45.826000,11.706000,6.464000,10.541000,283.270000
75%,13802.500000,2011-03-15 00:00:00,1.474000,55.880000,16.587500,9.118000,15.327000,287.240500
max,18403.000000,2012-12-31 00:00:00,2.841000,105.693000,60.627000,45.846000,76.765000,297.209000
std,5312.632838,NaN,0.304161,16.217363,7.419801,5.263998,6.921296,5.182186


In [3]:
# alternative way of checking dataset columns

print ("The train data has",train.shape)
print ("The test data has",test.shape)


The train data has (18403, 9)
The test data has (7886, 8)


In [6]:
# see what the data looks like

train.head(3000)

,Id,region,date,mortality_rate,O3,PM10,PM25,NO2,T2M
0,1,E12000001,2007-01-02,2.264,42.358,9.021,NaN,NaN,278.138
1,2,E12000001,2007-01-03,2.030,49.506,5.256,NaN,NaN,281.745
2,3,E12000001,2007-01-04,1.874,51.101,4.946,NaN,NaN,280.523
3,4,E12000001,2007-01-05,2.069,47.478,6.823,NaN,NaN,280.421
4,5,E12000001,2007-01-06,1.913,45.226,7.532,NaN,NaN,278.961
...,...,...,...,...,...,...,...,...,...
2995,2996,E12000009,2007-03-26,1.528,41.161,34.624,NaN,NaN,282.409
2996,2997,E12000009,2007-03-27,1.934,44.191,39.707,NaN,NaN,282.841
2997,2998,E12000009,2007-03-28,1.334,53.291,49.294,NaN,NaN,281.632
2998,2999,E12000009,2007-03-29,1.644,72.903,10.882,NaN,NaN,279.672


In [7]:
train.shape[0]

18403

In [8]:
# check if we have missing values in our data

nans = train.shape[0] - train.dropna().shape[0]
print ("%d rows have missing values in the train data" %nans)

nand = test.shape[0] - test.dropna().shape[0]
print ("%d rows have missing values in the test data" %nand)

6570 rows have missing values in the train data
0 rows have missing values in the test data


In [9]:
# Now let's check which columns have missing values
train.isnull().sum()

Id                   0
region               0
date                 0
mortality_rate       0
O3                   9
PM10                 9
PM25              3276
NO2               6570
T2M                  0
dtype: int64

In [10]:
# How many unique values does each column hold
categories = train.select_dtypes(include=['O'])

categories.apply(pd.Series.nunique)

region       9
date      2191
dtype: int64

#### Dealing with missing values

 - we identified that we had four columns where some values were missing
 - these columns affect our records, but we don't want to scap or lose them completely!
 - Let's impute these missing values with respective models
 - O3, PM10, NO2 - we imputed missing values using mean. 
 - PM25 and NO2 have a larger number of missing values. We don't want to  remove rows with missing data because we would loose data for all 2007 and 2008 . We will calculate mean for each month in the following years



In [11]:
#Convert date to date object
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

#Group the data by month and calculate the average pollutant concentration for each month during 2008-2012.
relevant_years_PM25 = train[(train['date'].dt.year >= 2008) & (train['date'].dt.year <= 2012)]
pm25_monthly_avg = relevant_years_PM25.groupby(relevant_years_PM25['date'].dt.month)['PM25'].mean().round(3)
# print('PM25', pm25_monthly_avg)

# impute missing values accordingly to avg for each month in 2007 

for index, row in train.iterrows():
    if pd.isnull(row['PM25']) and row['date'].year == 2007:
        month = row['date'].month
        train.at[index, 'PM25'] = pm25_monthly_avg[month]

# print(train['PM25'])

relevant_years_NO2 = train[(train['date'].dt.year >= 2009) & (train['date'].dt.year <= 2012)]
no2_monthly_avg = relevant_years_NO2.groupby(relevant_years_NO2['date'].dt.month)['NO2'].mean().round(3)
# print('no2',no2_monthly_avg)

# Check if PM25 values were imputed
print("Missing PM25 values after imputation for 2007:")
print(train[train['date'].dt.year == 2007]['PM25'].isnull().sum())

# impute missing values accordingly to avg for each month in 2007 and 2008

for index, row in train.iterrows():
    if pd.isnull(row['NO2']) and row['date'].year in [2007,2008]:
        month = row['date'].month
        train.at[index, 'NO2'] = no2_monthly_avg[month]

# Check if NO2 values were imputed
print("Missing NO2 values after imputation for 2007 and 2008:")
print(train[(train['date'].dt.year == 2007) | (train['date'].dt.year == 2008)]['NO2'].isnull().sum())

# Final check for missing data
print("Missing values after imputation:")
print(train.isnull().sum())

Missing PM25 values after imputation for 2007:
0
Missing NO2 values after imputation for 2007 and 2008:
0
Missing values after imputation:
Id                0
region            0
date              0
mortality_rate    0
O3                9
PM10              9
PM25              0
NO2               0
T2M               0
dtype: int64


In [13]:
# Calculate mean for O3, PM10

mean_O3 = train['O3'].mean()

print(mean_O3)

mean_PM10 = train['PM10'].mean()

print(mean_PM10)


# Impute those values 

train.value_counts('O3', sort= True)
train.fillna({'O3':mean_O3}, inplace= True)

train.value_counts('PM10', sort=True)
train.fillna({'PM10':mean_PM10}, inplace=True)

# Final check for missing data
print("Missing values after imputation:")
print(train.isnull().sum())


45.325857290420785
13.71227215396325
Missing values after imputation:
Id                0
region            0
date              0
mortality_rate    0
O3                0
PM10              0
PM25              0
NO2               0
T2M               0
dtype: int64


In [14]:
train.shape[0]

18403

In [12]:
# Check proportion of target variable
print(train.target.value_counts() / train.shape[0])

# Check for any remaining missing values in the dataset
print(train.isnull().sum())


AttributeError: 'DataFrame' object has no attribute 'target'

In [ ]:
# Let's check the 'target' variable (values in the 'target' column) to investigate if this data is imbalanced or not.

# check proportions % of target variable (there are only 2 unique values, 
# so we should get a % split of these values in our dataset)
train.target.value_counts()/train.shape[0]

target
<=50K    0.75919
>50K     0.24081
Name: count, dtype: float64

<div class="alert alert-block alert-info">
<b>75% of the data set belongs to less than 50K class. This means that even if we take a rough guess of target prediction as less than 50K, we'll get 75% accuracy.</b>
Let's create a cross tab of the target variable with education. With this, we'll try to understand the influence of education on the target variable.
</div>

In [ ]:
pd.crosstab(train.education, train.target,margins=True)/train.shape[0]

target,<=50K,>50K,All
education,,,
10th,0.026750,0.001904,0.028654
11th,0.034243,0.001843,0.036086
12th,0.012285,0.001013,0.013298
1st-4th,0.004975,0.000184,0.005160
5th-6th,0.009736,0.000491,0.010227
7th-8th,0.018611,0.001228,0.019840
9th,0.014957,0.000829,0.015786
Assoc-acdm,0.024631,0.008139,0.032769
Assoc-voc,0.031357,0.011087,0.042443




### FINDINGS: 

__out of 75% people with <=50K salary:__ 

- 27% people are high school graduates, which is correct as people with lower levels of education are expected to earn less. 

__out of 25% people with >=50K salary:__ 

- 6% are bachelors and 5% are high-school grads. ==> _this pattern seems to be a matter of concern! We have to consider more variables before coming to a conclusion._




## SCIKIT

- Let's try to utilise the mighty SciKit library for the next step. 

- __IMPORTANT: Scikit accepts data in numeric format! It means that we need to convert the character variables into numeric.__  


- To do our conversion, we will use the _labelencoder_ function.

- __IMPORTANT: In label encoding, each unique value of a variable gets assigned a number.__ Example: a variable fruit has four values: 'apple', 'banana', 'kiwi', 'melon'. Label encoding this variable will return output as: apple = 2 banana = 0 kiwi = 1 melon = 3

In [ ]:
# let's encode all object type variables

from sklearn import preprocessing
label_encoders = {}

for x in train.columns:
    if train[x].dtype == 'object':
        lbl = preprocessing.LabelEncoder() # encoding
        lbl.fit(list(train[x].values)) # fitting the model
        train[x] = lbl.transform(list(train[x].values))
        label_encoders[x] = lbl

In [ ]:
# mapping the values to variables from the above

for col, le in label_encoders.items():
    print(f"Column: {col}")
    for idx, class_ in enumerate(le.classes_):
        print(f"{class_} -> {idx}")

Column: workclass
 Federal-gov -> 0
 Local-gov -> 1
 Never-worked -> 2
 Private -> 3
 Self-emp-inc -> 4
 Self-emp-not-inc -> 5
 State-gov -> 6
 Without-pay -> 7
Private -> 8
Column: education
 10th -> 0
 11th -> 1
 12th -> 2
 1st-4th -> 3
 5th-6th -> 4
 7th-8th -> 5
 9th -> 6
 Assoc-acdm -> 7
 Assoc-voc -> 8
 Bachelors -> 9
 Doctorate -> 10
 HS-grad -> 11
 Masters -> 12
 Preschool -> 13
 Prof-school -> 14
 Some-college -> 15
Column: marital.status
 Divorced -> 0
 Married-AF-spouse -> 1
 Married-civ-spouse -> 2
 Married-spouse-absent -> 3
 Never-married -> 4
 Separated -> 5
 Widowed -> 6
Column: occupation
 Adm-clerical -> 0
 Armed-Forces -> 1
 Craft-repair -> 2
 Exec-managerial -> 3
 Farming-fishing -> 4
 Handlers-cleaners -> 5
 Machine-op-inspct -> 6
 Other-service -> 7
 Priv-house-serv -> 8
 Prof-specialty -> 9
 Protective-serv -> 10
 Sales -> 11
 Tech-support -> 12
 Transport-moving -> 13
Prof-specialty -> 14
Column: relationship
 Husband -> 0
 Not-in-family -> 1
 Other-relative -> 

In [ ]:
# examine the dataset again to see how our encoding changes have been applied

train.head(20)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target
0,39,6,77516,9,13,4,0,1,4,1,2174,0,40,38,0
1,50,5,83311,9,13,2,3,0,4,1,0,0,13,38,0
2,38,3,215646,11,9,0,5,1,4,1,0,0,40,38,0
3,53,3,234721,1,7,2,5,0,2,1,0,0,40,38,0
4,28,3,338409,9,13,2,9,5,2,0,0,0,40,4,0
5,37,3,284582,12,14,2,3,5,4,0,0,0,40,38,0
6,49,3,160187,6,5,3,7,1,2,0,0,0,16,22,0
7,52,5,209642,11,9,2,3,0,4,1,0,0,45,38,1
8,31,3,45781,12,14,4,9,1,4,0,14084,0,50,38,1
9,42,3,159449,9,13,2,3,0,4,1,5178,0,40,38,1


### CLOSING REMARKS

1. It was our first step towards familiarising ourselves with the world of Machine Learning.
1. Hopeully you found it interesting!
1. Next time we will cover more individual examples of ML processing and SciKit library. 
1. Remember, it takes months and sometimes years to build, fit and validate solid ML models. We won't be able to complete the full cycle, but we will focus on key concepts, preprocessing and visualisation ¯\_(ツ)_/¯

<img src="images/remarks.jpeg" style="width:200px;" />